In [75]:
ansible-playbook role.yml --tags "G1"


PLAY [Test role] ***************************************************************

TASK [[common/tasks/main] show task param] *************************************
ok: [debian1] => {
    "msg": "Param common_param=Common"
}

TASK [[setup/tasks/main]: Setup facts variables] *******************************
ok: [debian1]

TASK [[setup/tasks/main]: Check facts variables is set] ************************
ok: [debian1] => {
    "msg": "database={'name': 'mysql', 'url': 'mysql://localhost:3306/test-db'}"
}

TASK [[role1/tasks/main]: Show paramters and variables] ************************
ok: [debian1] => {
    "msg": [
        "Parameter main_param=Main",
        "Facts database=mysql and url=mysql://localhost:3306/test-db"
    ]
}

TASK [[role1/tasks/main, role1/vars/main]: Load variables] *********************
ok: [debian1] => {
    "msg": "a=100, b=200’"
}

TASK [[role1/task/main, role1/templates/db.conf.t]: Template transfer] *********
changed: [debian1]

TASK [[role1/task/main, role1/templ